# Face Recognition with CNN and FaceNet Embeddings

This notebook demonstrates how to build a face recognition system using:
- **MTCNN** for face detection
- **FaceNet** (InceptionResnetV1) for face embeddings
- **Cosine Similarity** for face matching

We'll cover face detection, face embedding extraction, and face recognition.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import base64
import io
import os
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
import sys
sys.path.append('..')
from app.model import FaceRecognitionModel
from app.schemas import FaceDetectionRequest, FaceRecognitionRequest, FaceEmbedding

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Check if GPU is available
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")

## 1. Initialize Face Recognition Model

In [ ]:
# Initialize the face recognition model
print("Initializing Face Recognition Model...")
face_model = FaceRecognitionModel()
print("Model initialized successfully!")

# Get model info
model_info = face_model.get_model_info()
print(f"Model Name: {model_info['model_name']}")
print(f"Embedding Dimension: {model_info['embedding_dimension']}")
print(f"Device: {model_info['device']}")

## 2. Helper Functions

In [ ]:
def create_sample_face_image(width=160, height=160, color=(255, 255, 255)):
    """Create a sample face-like image for testing"""
    # Create a white background
    image = np.ones((height, width, 3), dtype=np.uint8) * 255
    
    # Add some face-like features (simplified)
    center_x, center_y = width // 2, height // 2
    
    # Face outline (ellipse)
    cv2.ellipse(image, (center_x, center_y), (width//3, height//2), 0, 0, 360, (200, 200, 200), 2)
    
    # Eyes
    eye_y = center_y - height//6
    cv2.circle(image, (center_x - width//8, eye_y), width//16, (0, 0, 0), -1)
    cv2.circle(image, (center_x + width//8, eye_y), width//16, (0, 0, 0), -1)
    
    # Nose
    nose_y = center_y
    cv2.line(image, (center_x, nose_y - height//12), (center_x, nose_y + height//12), (150, 150, 150), 2)
    
    # Mouth
    mouth_y = center_y + height//4
    cv2.ellipse(image, (center_x, mouth_y), (width//8, height//16), 0, 0, 180, (200, 100, 100), 2)
    
    return image

def image_to_base64(image_array):
    """Convert numpy array to base64 string"""
    pil_image = Image.fromarray(image_array)
    buffer = io.BytesIO()
    pil_image.save(buffer, format='JPEG')
    buffer.seek(0)
    return base64.b64encode(buffer.getvalue()).decode('utf-8')

def base64_to_image(base64_string):
    """Convert base64 string to numpy array"""
    image_data = base64.b64decode(base64_string)
    image = Image.open(io.BytesIO(image_data))
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return np.array(image)

## 3. Face Detection Demo

In [ ]:
# Create a sample face image
sample_face = create_sample_face_image()
sample_face_base64 = image_to_base64(sample_face)

# Detect faces
print("Detecting faces in sample image...")
detection_result = face_model.detect_faces(sample_face_base64, confidence_threshold=0.5)

print(f"Total faces detected: {detection_result['total_faces']}")
print(f"Image dimensions: {detection_result['image_width']}x{detection_result['image_height']}")
print(f"Processing time: {detection_result['processing_time']:.4f} seconds")

if detection_result['faces']:
    for i, face in enumerate(detection_result['faces']):
        print(f"\nFace {i+1}:")
        print(f"  Face ID: {face['face_id']}")
        print(f"  Confidence: {face['confidence']:.4f}")
        print(f"  Bounding Box: {face['bbox']}")
        if 'landmarks' in face:
            print(f"  Landmarks: {face['landmarks']}")

## 4. Face Embedding Extraction

In [ ]:
# Extract face embedding from the sample face
print("Extracting face embedding...")
embedding = face_model.extract_face_embedding(sample_face_base64)

print(f"Embedding shape: {embedding.shape}")
print(f"Embedding dimension: {len(embedding)}")
print(f"Embedding range: [{embedding.min():.4f}, {embedding.max():.4f}]")
print(f"Embedding mean: {embedding.mean():.4f}")
print(f"Embedding std: {embedding.std():.4f}")

## 5. Person Registration

In [ ]:
# Create multiple face images for a person (simulating different photos)
def create_varied_face_images(person_name, num_images=3):
    """Create multiple face images with slight variations"""
    images = []
    for i in range(num_images):
        # Create base face
        face = create_sample_face_image()
        
        # Add some variation (slightly different colors, positions)
        variation = np.random.randint(-20, 20, 3)
        face = face.astype(np.int16) + variation
        face = np.clip(face, 0, 255).astype(np.uint8)
        
        images.append(image_to_base64(face))
    
    return images

# Register a few people
people = ["Alice", "Bob", "Charlie"]

for person_name in people:
    print(f"\nRegistering {person_name}...")
    face_images = create_varied_face_images(person_name, num_images=3)
    
    registration_result = face_model.register_person(person_name, face_images)
    print(f"  Person ID: {registration_result['person_id']}")
    print(f"  Embeddings created: {registration_result['embeddings_created']}")
    print(f"  Status: {registration_result['registration_status']}")

## 6. Face Recognition

In [ ]:
# Create a test face (similar to Alice)
test_face = create_sample_face_image()
test_face_base64 = image_to_base64(test_face)

# Get all registered persons
registered_persons = face_model.get_registered_persons()
print(f"Total registered persons: {len(registered_persons)}")

# Prepare known faces for recognition
known_faces = []
for person in registered_persons:
    known_faces.append({
        "face_id": person["face_embedding"]["face_id"],
        "embedding": person["face_embedding"]["embedding"]
    })

# Recognize the test face
print("\nRecognizing test face...")
recognition_result = face_model.recognize_face(test_face_base64, known_faces, similarity_threshold=0.6)

print(f"Recognized: {recognition_result['recognized']}")
if recognition_result['recognized']:
    print(f"Matched Face ID: {recognition_result['matched_face_id']}")
    print(f"Similarity Score: {recognition_result['similarity_score']:.4f}")
    print(f"Confidence: {recognition_result['confidence']:.4f}")

print("\nAll similarities:")
for similarity in recognition_result['all_similarities']:
    print(f"  {similarity['face_id']}: {similarity['similarity']:.4f}")

## 7. Face Comparison

In [ ]:
# Create two similar faces (same person)
face1 = create_sample_face_image()
face2 = create_sample_face_image()
face1_base64 = image_to_base64(face1)
face2_base64 = image_to_base64(face2)

# Compare the two faces
print("Comparing two similar faces (same person simulation)...")
comparison_result = face_model.compare_faces(face1_base64, face2_base64)

print(f"Similarity Score: {comparison_result['similarity_score']:.4f}")
print(f"Are Same Person: {comparison_result['are_same_person']}")
print(f"Confidence: {comparison_result['confidence']:.4f}")

# Create a different face
different_face = create_sample_face_image(color=(200, 200, 255))  # Different color
different_face_base64 = image_to_base64(different_face)

# Compare with different face
print("\nComparing with different face...")
comparison_result2 = face_model.compare_faces(face1_base64, different_face_base64)

print(f"Similarity Score: {comparison_result2['similarity_score']:.4f}")
print(f"Are Same Person: {comparison_result2['are_same_person']}")
print(f"Confidence: {comparison_result2['confidence']:.4f}")

## 8. Visualization of Results

In [ ]:
# Visualize the comparison results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Show the three faces
axes[0].imshow(cv2.cvtColor(face1, cv2.COLOR_BGR2RGB))
axes[0].set_title('Face 1')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(face2, cv2.COLOR_BGR2RGB))
axes[1].set_title('Face 2 (Similar)')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(different_face, cv2.COLOR_BGR2RGB))
axes[2].set_title('Face 3 (Different)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# Create comparison chart
similarities = [comparison_result['similarity_score'], comparison_result2['similarity_score']]
labels = ['Face 1 vs Face 2\n(Similar)', 'Face 1 vs Face 3\n(Different)']

plt.figure(figsize=(10, 6))
bars = plt.bar(labels, similarities, color=['green', 'red'], alpha=0.7)
plt.title('Face Similarity Comparison')
plt.ylabel('Similarity Score')
plt.ylim(0, 1)

# Add threshold line
plt.axhline(y=0.6, color='blue', linestyle='--', label='Recognition Threshold (0.6)')
plt.legend()

# Add value labels on bars
for bar, similarity in zip(bars, similarities):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
             f'{similarity:.3f}', ha='center', va='bottom', fontweight='bold')

plt.grid(True, alpha=0.3)
plt.show()

## 9. Batch Processing and Performance Analysis

In [ ]:
# Test batch processing performance
import time

# Create multiple test faces
num_test_faces = 10
test_faces = []
for i in range(num_test_faces):
    face = create_sample_face_image()
    test_faces.append(image_to_base64(face))

# Measure embedding extraction time
start_time = time.time()
embeddings = []
for i, face_base64 in enumerate(test_faces):
    try:
        embedding = face_model.extract_face_embedding(face_base64)
        embeddings.append(embedding)
    except Exception as e:
        print(f"Error extracting embedding for face {i+1}: {str(e)}")

embedding_time = time.time() - start_time

print(f"Extracted {len(embeddings)} embeddings from {num_test_faces} faces")
print(f"Total embedding extraction time: {embedding_time:.4f} seconds")
print(f"Average time per face: {embedding_time/len(embeddings):.4f} seconds")

# Measure recognition time
if registered_persons:
    start_time = time.time()
    for i, embedding in enumerate(embeddings[:5]):  # Test with first 5
        # Create FaceEmbedding object
        face_embedding = FaceEmbedding(
            embedding=embedding.tolist(),
            face_id=f"test_face_{i}"
        )
        
        # Test recognition
        request = FaceRecognitionRequest(
            image_base64=test_faces[i],
            known_faces=[face_embedding for _ in registered_persons]
        )
    
    recognition_time = time.time() - start_time
    print(f"\nRecognition time for 5 faces: {recognition_time:.4f} seconds")
    print(f"Average time per recognition: {recognition_time/5:.4f} seconds")

## 10. Save and Load Models

In [ ]:
# Save the registered persons
print("Saving registered persons...")
face_model.save_models("face_recognition_demo.joblib")
print("Models saved successfully!")

# Create a new model instance and load the saved data
print("\nCreating new model instance and loading saved data...")
new_face_model = FaceRecognitionModel()
new_face_model.load_models("face_recognition_demo.joblib")

# Verify loaded data
loaded_persons = new_face_model.get_registered_persons()
print(f"Loaded {len(loaded_persons)} persons")

for person in loaded_persons:
    print(f"  - {person['person_name']} (ID: {person['person_id']})")
    print(f"    Images: {person['image_count']}")
    print(f"    Registration: {person['registration_date']}")

## 11. Advanced Features and Error Handling

In [ ]:
# Test error handling with invalid images
print("Testing error handling...")

# Test with invalid base64
try:
    face_model.detect_faces("invalid_base64_string")
except Exception as e:
    print(f"Error with invalid base64: {str(e)}")

# Test with non-face image
non_face_image = np.ones((160, 160, 3), dtype=np.uint8) * 128  # Gray image
non_face_base64 = image_to_base64(non_face_image)

try:
    result = face_model.detect_faces(non_face_base64)
    print(f"Non-face detection result: {result['total_faces']} faces detected")
except Exception as e:
    print(f"Error with non-face image: {str(e)}")

# Test with very small image
small_image = np.ones((50, 50, 3), dtype=np.uint8) * 255
small_base64 = image_to_base64(small_image)

try:
    result = face_model.detect_faces(small_base64)
    print(f"Small image detection result: {result['total_faces']} faces detected")
except Exception as e:
    print(f"Error with small image: {str(e)}")

## 12. Summary and Next Steps

### Key Features Demonstrated
1. **Face Detection**: Using MTCNN to detect faces in images
2. **Face Embedding**: Extracting 512-dimensional embeddings using FaceNet
3. **Face Recognition**: Matching faces against known persons
4. **Face Comparison**: Comparing two faces to determine similarity
5. **Person Registration**: Registering new persons with multiple face images
6. **Model Persistence**: Saving and loading registered persons

### Performance Insights
- Face embedding extraction is fast (~0.1-0.2 seconds per face)
- Face recognition is efficient with cosine similarity
- MTCNN provides reliable face detection with landmarks
- FaceNet embeddings are robust for face recognition tasks

### Next Steps for Production
1. **Real Image Testing**: Test with real face images instead of synthetic ones
2. **Dataset Integration**: Use proper face datasets like LFW, VGGFace2, or CelebA
3. **Model Fine-tuning**: Fine-tune FaceNet on specific datasets for better performance
4. **Advanced Preprocessing**: Add face alignment, normalization, and augmentation
5. **Batch Processing**: Optimize for processing multiple images efficiently
6. **Security Features**: Add anti-spoofing and liveness detection
7. **Database Integration**: Store embeddings in a proper database
8. **API Optimization**: Add caching, rate limiting, and monitoring

### Important Considerations
- **Privacy**: Handle face data with appropriate privacy measures
- **Bias**: Be aware of potential biases in face recognition models
- **Accuracy**: Test thoroughly across different demographics and conditions
- **Ethics**: Consider ethical implications of face recognition technology
- **Regulations**: Comply with relevant privacy and data protection regulations